In [ ]:
import gc
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import  SGD, RMSprop, Adam
from functools import cmp_to_key
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new')

edges = [[1, 2], [1, 5], [1, 8], [2, 3], [3, 4], [3, 8], [3, 9], [4, 10], [5, 6], [6, 7], [6, 8], [6, 9], [7, 10], [9, 10]]
number_edges = len(edges)
number_steps = 4
start_node = 1
end_node = 10

df = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU.csv')
x = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

df_hp = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU_hp.csv')
x_hp = df_hp.iloc[:,:number_edges].values
y_hp = df_hp.iloc[:,number_edges:].values

from sklearn.model_selection import train_test_split
x_main_train,x_main_tv,y_main_train,y_main_tv = train_test_split(x,y,test_size=2/5)
x_main_test,x_main_valid,y_main_test,y_main_valid = train_test_split(x_main_tv,y_main_tv,test_size=1/2)

x_hp_train,x_hp_tv,y_hp_train,y_hp_tv = train_test_split(x_hp,y_hp,test_size=2/5)
x_hp_test,x_hp_valid,y_hp_test,y_hp_valid = train_test_split(x_hp_tv,y_hp_tv,test_size=1/2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_main_train)
x_test = sc.transform(x_main_test)
x_valid = sc.transform(x_main_valid)

sc_hp = StandardScaler()
x_train_1 = sc_hp.fit_transform(x_hp_train)
x_test_1 = sc_hp.transform(x_hp_test)
x_valid_1 = sc_hp.transform(x_hp_valid)

x_train = np.repeat(x_train[:, np.newaxis, : ], number_steps, axis=1)
x_test = np.repeat(x_test[:, np.newaxis, : ], number_steps, axis=1)
x_valid = np.repeat(x_valid[:, np.newaxis, : ], number_steps, axis=1)

x_train_1 = np.repeat(x_train_1[:, np.newaxis, : ], number_steps, axis=1)
x_test_1 = np.repeat(x_test_1[:, np.newaxis, : ], number_steps, axis=1)
x_valid_1 = np.repeat(x_valid_1[:, np.newaxis, : ], number_steps, axis=1)

y_train = np.reshape(y_main_train,(y_main_train.shape[0], number_steps, number_edges))
y_test = np.reshape(y_main_test,(y_main_test.shape[0], number_steps, number_edges))
y_valid = np.reshape(y_main_valid,(y_main_valid.shape[0], number_steps, number_edges))

y_train_1 = np.reshape(y_hp_train,(y_hp_train.shape[0], number_steps, number_edges))
y_test_1 = np.reshape(y_hp_test,(y_hp_test.shape[0], number_steps, number_edges))
y_valid_1 = np.reshape(y_hp_valid,(y_hp_valid.shape[0], number_steps, number_edges))

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape, x_train_1.shape, y_train_1.shape, x_test_1.shape, y_test_1.shape)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,"X(1, 2)","X(1, 5)","X(1, 8)","X(2, 3)","X(3, 4)","X(3, 8)","X(3, 9)","X(4, 10)","X(5, 6)","X(6, 7)",...,"Y4(3, 4)","Y4(3, 8)","Y4(3, 9)","Y4(4, 10)","Y4(5, 6)","Y4(6, 7)","Y4(6, 8)","Y4(6, 9)","Y4(7, 10)","Y4(9, 10)"
0,40,30,90,60,80,10,80,40,20,90,...,0,1,0,0,0,0,1,1,0,1
1,80,50,80,70,70,80,10,90,60,90,...,0,0,0,0,1,0,0,1,0,1
2,50,90,20,30,10,30,60,60,50,30,...,0,0,0,0,0,0,1,1,0,1
3,10,80,30,80,20,60,20,40,30,10,...,0,0,1,0,0,0,0,0,0,1
4,90,70,90,60,80,20,60,20,60,40,...,0,1,1,0,0,0,0,0,0,1


In [ ]:
# Hyperparameters of structure: number of layers, number of units, dropout, cell type
# Hyperparameters of training: learning rate, batch size
HP_1 = [[1, 2, 3, 4, 5],
       [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, -1],
       [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
       [SimpleRNN, GRU, LSTM],
       [SGD, RMSprop, Adam],
       [0.0001, 0.001, 0.01, 0.1],
       [16, 32, 64, 128, 256, 512, 1024]]

In [ ]:
t = []
for p in HP_1:
    t.append(len(p))
t1 = []
hp = []
for i in range(t[0]):
    t1.append(t[1])
    t1.append(t[2])
    hp.append(HP_1[1])
    hp.append(HP_1[2])
for i in range(3, len(HP_1)):
    t1.append(t[i])
    hp.append(HP_1[i])
t2 = [int(np.sqrt(itm)) for itm in t1]
n_hp = 2*t[0] + len(HP_1) - 3
print(t, t1, t2, n_hp)

[5, 22, 5, 4, 5, 5, 5, 7]

In [ ]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, ratio=0.0, patience=0, verbose=0):
        super(CustomEarlyStopping, self).__init__()

        self.ratio = ratio
        self.patience = patience
        self.verbose = verbose
        self.stopped_epoch = 0
        self.val_loss_1 = 0
        self.rt = 0

    def on_epoch_end(self, epoch, logs=None):
        if epoch == 1:
            self.val_loss_1 = logs.get('val_loss')
        if epoch == self.patience:
            current_val = logs.get('val_loss')
            if current_val/self.val_loss_1 > self.ratio:
                self.rt = current_val/self.val_loss_1
                self.stopped_epoch = epoch
                self.model.stop_training = True

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0 and self.verbose > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch))
            print("Current ratio = ", self.rt)

In [ ]:
class Network:
    def __init__(self):
        self.gen = []
        self.fitness = float("inf")
        self.delta = 0

    def check_gen(self):
        state = False
        for i in range(n_hp):
            if i%2 == 0 and i < 2*t[0] and hp[i][self.gen[i]] != -1:
                state = True
                break
        return state

    def evaluate(self):
        if self.check_gen() == False:
            return float("inf")
        else:
            input_shape = (number_steps, number_edges)
            inputs = Input(shape=input_shape)
            x_input = inputs
            cell_type = hp[2*t[0]][self.gen[2*t[0]]]
            for i in range(t[0]):
                if hp[2*i][self.gen[2*i]] != -1:
                    x_input = cell_type(hp[2*i][self.gen[2*i]], return_sequences=True)(x_input)
                    x_input = Dropout(hp[2*i+1][self.gen[2*i+1]])(x_input)
            outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
            model = Model(inputs=inputs, outputs=outputs)
            opt = hp[2*t[0]+1][self.gen[2*t[0]+1]]
            model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][self.gen[2*t[0]+2]]))
            callback_1 = CustomEarlyStopping(ratio=0.9, patience=10, verbose=1)
            callback_2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
            model.fit(x_train_1, y_train_1, epochs=100, callbacks=[callback_1, callback_2], validation_data=(x_valid_1, y_valid_1), batch_size=hp[2*t[0]+3][self.gen[2*t[0]+3]], verbose=0)
            loss = model.evaluate(x_test_1, y_test_1, verbose=0)
            tf.keras.backend.clear_session()
            return loss

In [ ]:
class AS:
    def __init__(self, N, Max, p, a, b, Q):
        self.N = N
        self.Max = Max
        self.p = p
        self.a = a
        self.b = b
        self.Q = Q
        self.colony = [Network() for i in range(self.N)]
        self.best = Network()
        self.pool = []
        # self.Lnn = self.find_Lnn()
        self.t0 = 1/n_hp
        self.pheromone = self.create_pheromone()

    def colony_fitness(self):
        for i in range(len(self.colony)):
            self.calculate_fitness(self.colony[i])
            self.show(self.colony[i])
        print("__________________________________________________________________")

    def calculate_fitness(self, new_mem):
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):
                cond = False
                new_mem.fitness = mem.fitness
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()
            self.pool.append(copy.deepcopy(new_mem))

    def complexity(self, m):
        coeff = None
        r_a = 0
        if m.gen[2*t[0]] == 0:
            coeff = 1
        elif m.gen[2*t[0]] == 1:
            coeff = 3
            r_a = 1
        else:
            coeff = 4
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2*i][m.gen[2*i]] != -1:
                b = coeff*hp[2*i][m.gen[2*i]]*(hp[2*i][m.gen[2*i]] + g + r_a + 1)
                a += b
                g = hp[2*i][m.gen[2*i]]
        a += number_edges*(g+1)
        return a

    def compare(self, m1, m2):
        if abs(m1.fitness-m2.fitness) < 0.01*min(m1.fitness, m2.fitness):
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                if m1.fitness < m2.fitness:
                    return -1
                elif m1.fitness == m2.fitness:
                    return 0
                else:
                    return 1
            else:
                return 1
        else:
            if m1.fitness < m2.fitness:
                return -1
            else:
                return 1

    # def find_Lnn(self):
    #     new_n = Network()
    #     for pr in range(n_hp):
    #         pos_pr = self.get_next_parameter_Lnn(pr)
    #         new_n.gen.append(pos_pr)
    #     self.calculate_fitness(new_n)
    #     return new_n.fitness

    # def get_next_parameter_Lnn(self, pr):
    #     next_value = 0
    #     for i in range(t1[pr]):
    #         if hp[pr][next_value] > hp[pr][i]:
    #             next_value = i
    #     return next_value

    def create_pheromone(self):
        pheromone = copy.deepcopy(hp)
        for i in range(n_hp):
            for j in range(t1[i]):
                pheromone[i][j] = self.t0
        return pheromone

    def create_path(self):
        for i in range(self.N):
            gen = []
            for pr in range(n_hp):
                pos_pr = self.get_next_parameter(pr)
                gen.append(pos_pr)
            self.colony[i].gen = copy.deepcopy(gen)
            self.calculate_fitness(self.colony[i])
            self.colony[i].delta = self.Q/(self.colony[i].fitness*n_hp)
            self.show(self.colony[i])
            print("__________________________________________________________________")

    def get_next_parameter(self, pr):
        total_prob = 0
        prob = []
        for i in range(t1[pr]):
            x = self.pheromone[pr][i]
            y = 1/t1[pr]
            z = pow(x,self.a)*pow(y,self.b)
            total_prob += z
            prob.append(z)
        prob = [prb/total_prob for prb in prob]
        pos_pr = np.random.choice(list(range(t1[pr])), p=prob)
        return pos_pr

    def update_pheromone(self):
        for i in range(n_hp):
            for j in range(t1[i]):
                self.pheromone[i][j] = self.pheromone[i][j]*(1-self.p)
        for i in range(self.N):
            for j in range(n_hp):
                j2 = self.colony[i].gen[j]
                self.pheromone[j][j2] += self.colony[i].delta

    def current_colony(self):
        for i in range(len(self.colony)):
            self.show(self.colony[i])
        print("__________________________________________________________________")

    def get_best(self):
        self.colony.sort(key=cmp_to_key(self.compare))
        self.current_colony()
        if self.compare(self.best, self.colony[0]) == 1:
            self.best = copy.deepcopy(self.colony[0])
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2*j], m.gen[2*j+1]))
        p_copy.append(m.gen[2*t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        for itr in range(self.Max):
            print("iteration = ", itr)
            print(self.pheromone[10])
            self.create_path()
            self.update_pheromone()
            self.get_best()
            gc.collect()

In [ ]:
N = 10
Max = 10
p = 0.1
a = 1
b = 1
Q = 1

In [ ]:
alg = AS(N, Max, p, a, b, Q)

Epoch 00010: early stopping
Current ratio =  0.9298093851956215


In [ ]:
import time
start = time.time()
alg.do()
end = time.time()

iteration =  0
Epoch 8: early stopping
[(8, 2), (5, 0), (4, 1), (17, 3), (9, 3), [0, 1, 2, 2, 3]] 0.5869207382202148
__________________________________________________________________
Epoch 00010: early stopping
Current ratio =  0.8804400455254602
[(12, 3), (19, 4), (20, 3), (16, 1), (1, 4), [2, 2, 0, 3, 6]] 0.5980705618858337
__________________________________________________________________
Epoch 00010: early stopping
Current ratio =  0.864216570083051
[(0, 2), (6, 3), (15, 4), (5, 2), (21, 4), [0, 4, 4, 3, 5]] 0.4091707766056061
__________________________________________________________________
Epoch 00010: early stopping
Current ratio =  0.8636733954043391
[(16, 3), (16, 3), (8, 4), (13, 0), (21, 4), [2, 4, 2, 0, 4]] 0.5316130518913269
__________________________________________________________________
Epoch 00010: early stopping
Current ratio =  0.9524612452619009
[(6, 0), (1, 1), (9, 4), (15, 3), (19, 2), [3, 3, 2, 1, 0]] 0.5612567663192749
________________________________________

In [ ]:
alg.best.gen

In [ ]:
run_time = end - start
round(run_time)

In [ ]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
# input_shape = (number_steps, number_edges)
# inputs = Input(shape=input_shape)
# x_input = inputs
# cell_type = hp[2*t[0]][alg.best.gen[2*t[0]]]
# for i in range(t[0]):
#     if hp[2*i][alg.best.gen[2*i]] != -1:
#         x_input = cell_type(hp[2*i][alg.best.gen[2*i]], return_sequences=True)(x_input)
#         x_input = Dropout(hp[2*i+1][alg.best.gen[2*i+1]])(x_input)
# outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
# model = Model(inputs=inputs, outputs=outputs)
# opt = hp[2*t[0]+1][alg.best.gen[2*t[0]+1]]
# model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][alg.best.gen[2*t[0]+2]]))
# model.summary()
# print(alg.complexity(alg.best))
# print(opt)
# history = model.fit(x_train, y_train, epochs=1000, batch_size=hp[2*t[0]+3][alg.best.gen[2*t[0]+3]], callbacks=[callback], validation_data=(x_valid, y_valid), verbose=1)

In [ ]:
# from Results import Results
# pred = model.predict(x_train)
# pred_test = model.predict(x_test)
# r = Results(number_edges, edges, start_node, end_node, number_steps)
# accuracy_train = r.get_accuracy(pred, x_main_train, y_train)
# accuracy_test = r.get_accuracy(pred_test, x_main_test, y_test)
# print(accuracy_train, accuracy_test)